In [16]:
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import StochRSIIndicator
from ta.momentum import RSIIndicator
from ta.volume import ForceIndexIndicator
payload=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
#print(first_table.head())
sip=np.array(payload.iloc[:,0])
df=pd.DataFrame()
dfout=pd.DataFrame()
start_date="2000-01-01"
end_date="2021-10-01"
intr="1mo"
st_rsi_os=0.30
st_rsi_ob=0.70




#for i in range(0,len(sip)):
for i in range(0,1):
    stk=sip[i]
    df = yf.download(stk, start_date, end_date,interval=intr)
    df=df.dropna()

    stoch_rsi=StochRSIIndicator(close=df.Close,window=14,smooth1=3, smooth2=3)
    df["k"]=stoch_rsi.stochrsi_k().shift(1)
    
    for_ind=ForceIndexIndicator(close=df.Close,volume=df.Volume,window=13)
    df["fi"]=for_ind.force_index().shift(1)
    #for_ind_rsi=RSIIndicator(close=df.fi,window=14)
    #df["fi_k"]=for_ind_rsi
    
    df["ChngP"]=(df.Close-df.Open)/df.Open
    
                                   
    cond1=[
        (df.k<.30) & (df.fi <0),
        (df.k>.70) & (df.fi >0)]
    val1=[-1,1]
    df['chk']=np.select(cond1,val1)
    
    ovrs=df[df["chk"]==-1]
    ovrb=df[df["chk"]==1]
    ovrs_chng=np.average(ovrs.ChngP)
    ovrb_chng=np.average(ovrb.ChngP)
    ovrs_edg="{:.2%}".format(ovrs_chng-np.average(df.ChngP))
    ovrb_edg="{:.2%}".format(ovrb_chng-np.average(df.ChngP))
    temp=pd.Series([stk,ovrs_edg,ovrb_edg])
    dfout=dfout.append(temp,ignore_index=True)
print(dfout.tail())
#dfout.to_csv('stochrsi_mon_edge.csv')

[*********************100%***********************]  1 of 1 completed
     0       1       2
0  MMM  -0.02%  -0.18%


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 